In [1]:
# ------------------------------------------
# CELDA 1: Importar bibliotecas necesarias
# ------------------------------------------

import sqlite3  # Importamos la biblioteca para conectarnos a una base de datos SQLite
import csv      # Importamos csv para poder leer archivos CSV
import os       # Importamos os para manejar rutas y archivos en el sistema operativo

# Definimos las rutas de archivos en variables para facilidad de mantenimiento
# Ajusta estas rutas si tienes una estructura de carpetas diferente.
data_folder = "../data"        # Carpeta donde se ubican los archivos CSV
db_folder   = "../db"          # Carpeta donde se guardará la base de datos
db_name     = "production.db"  # Nombre del archivo de base de datos
print(f'Bibliotecas instaladas')

Bibliotecas instaladas


In [2]:
# ------------------------------------------
# CELDA 2: Crear/conectar la base de datos
# ------------------------------------------

# Verificamos si la carpeta db existe. Si no, la creamos.
if not os.path.exists(db_folder):
    os.makedirs(db_folder)  # Creamos la carpeta db si no existe

# Construimos la ruta completa a la base de datos
db_path = os.path.join(db_folder, db_name)

# Creamos la conexión a la base de datos SQLite
# Si el archivo production.db no existe, se creará automáticamente.
conn = sqlite3.connect(db_path)

# Creamos un cursor que nos permitirá ejecutar comandos SQL
cursor = conn.cursor()

print(f"Base de datos creada/conectada en: {db_path}")


Base de datos creada/conectada en: ../db\production.db


In [3]:
# ------------------------------------------
# CELDA 3: Crear tablas (orders y products)
# ------------------------------------------

# Eliminamos las tablas si existen (solo para reiniciar el ejemplo cada vez)
cursor.execute("DROP TABLE IF EXISTS orders;")   # Borramos la tabla 'orders' si existía
cursor.execute("DROP TABLE IF EXISTS products;") # Borramos la tabla 'products' si existía

# Creamos la tabla 'orders' con las columnas apropiadas
create_orders_table = """
CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    quantity INTEGER,
    order_date TEXT,
    status TEXT
);
"""

# Creamos la tabla 'products' con las columnas apropiadas
create_products_table = """
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    price REAL,
    category TEXT
);
"""

# Ejecutamos las sentencias de creación
cursor.execute(create_orders_table)
cursor.execute(create_products_table)

# Confirmamos los cambios
conn.commit()

print("Tablas 'orders' y 'products' creadas exitosamente.")


Tablas 'orders' y 'products' creadas exitosamente.


In [4]:
# ------------------------------------------
# CELDA 4: Cargar datos CSV a las tablas
# ------------------------------------------

# Definimos la ruta de cada archivo CSV
orders_csv_path = os.path.join(data_folder, "orders.csv")
products_csv_path = os.path.join(data_folder, "products.csv")

# Cargamos datos en la tabla 'orders'
with open(orders_csv_path, mode='r', encoding='utf-8-sig') as f:  # Abrimos el archivo CSV de orders
    reader = csv.DictReader(f)  # DictReader para acceder a las columnas por nombre
    for row in reader:
        # Extraemos cada valor (conversión de tipos si corresponde)
        order_id = int(row['order_id'])            # Convertimos a int
        product_id = int(row['product_id'])        # Convertimos a int
        quantity = int(row['quantity'])            # Convertimos a int
        order_date = row['order_date']             # Cadena de texto
        status = row['status']                     # Cadena de texto
        
        # Preparamos la sentencia INSERT con placeholders
        insert_order_query = """
        INSERT INTO orders (order_id, product_id, quantity, order_date, status)
        VALUES (?, ?, ?, ?, ?);
        """
        
        # Ejecutamos la inserción con la tupla de valores
        cursor.execute(insert_order_query, (order_id, product_id, quantity, order_date, status))

# Cargamos datos en la tabla 'products'
with open(products_csv_path, mode='r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    for row in reader:
        product_id = int(row['product_id'])
        product_name = row['product_name']
        price = float(row['price'])
        category = row['category']
        
        insert_product_query = """
        INSERT INTO products (product_id, product_name, price, category)
        VALUES (?, ?, ?, ?);
        """
        
        cursor.execute(insert_product_query, (product_id, product_name, price, category))

# Confirmamos los cambios
conn.commit()

print("Datos cargados correctamente desde CSV a las tablas.")


Datos cargados correctamente desde CSV a las tablas.


In [5]:
# ------------------------------------------
# CELDA 5: Consulta básica de verificación
# ------------------------------------------

# Ejemplo de SELECT * para mostrar todos los registros de la tabla 'orders'
select_all_orders = "SELECT * FROM orders;"
cursor.execute(select_all_orders)  # Ejecutamos la consulta
rows_orders = cursor.fetchall()    # Obtenemos todos los resultados en una lista

print("Tabla 'orders':")
for row in rows_orders:
    print(row)

print("\n--------------------\n")

# Ejemplo de SELECT * para la tabla 'products'
select_all_products = "SELECT * FROM products;"
cursor.execute(select_all_products)
rows_products = cursor.fetchall()

print("Tabla 'products':")
for row in rows_products:
    print(row)


Tabla 'orders':
(1, 101, 3, '2023-01-10', 'shipped')
(2, 103, 1, '2023-01-11', 'shipped')
(3, 102, 2, '2023-01-12', 'cancelled')
(4, 101, 5, '2023-01-15', 'shipped')
(5, 104, 2, '2023-01-15', 'pending')
(6, 105, 7, '2023-01-18', 'shipped')

--------------------

Tabla 'products':
(101, 'Widget A', 25.5, 'Category A')
(102, 'Widget B', 15.0, 'Category A')
(103, 'Thingamajig', 40.75, 'Category B')
(104, 'Gadget', 10.0, 'Category A')
(105, 'Machine Part', 50.0, 'Category C')


In [6]:
# ------------------------------------------
# CELDA 6: WHERE y ORDER BY
# ------------------------------------------

# 1. Seleccionar todos los pedidos con status = 'shipped'
query_shipped_orders = """
SELECT * 
FROM orders
WHERE status = 'shipped';
"""
cursor.execute(query_shipped_orders)
result_shipped = cursor.fetchall()
print("Pedidos con status = 'shipped':")
for row in result_shipped:
    print(row)

print("\n--------------------\n")

# 2. Seleccionar todos los productos ordenados por precio descendente
query_products_desc = """
SELECT * 
FROM products
ORDER BY price DESC;
"""
cursor.execute(query_products_desc)
result_products_desc = cursor.fetchall()
print("Productos ordenados por precio (DESC):")
for row in result_products_desc:
    print(row)


Pedidos con status = 'shipped':
(1, 101, 3, '2023-01-10', 'shipped')
(2, 103, 1, '2023-01-11', 'shipped')
(4, 101, 5, '2023-01-15', 'shipped')
(6, 105, 7, '2023-01-18', 'shipped')

--------------------

Productos ordenados por precio (DESC):
(105, 'Machine Part', 50.0, 'Category C')
(103, 'Thingamajig', 40.75, 'Category B')
(101, 'Widget A', 25.5, 'Category A')
(102, 'Widget B', 15.0, 'Category A')
(104, 'Gadget', 10.0, 'Category A')


In [7]:
# ------------------------------------------
# CELDA 7: GROUP BY y agregaciones
# ------------------------------------------

# Queremos saber cuántos pedidos (conteo) hay por cada producto_id
query_group_by_product = """
SELECT product_id, COUNT(*) AS total_orders
FROM orders
GROUP BY product_id;
"""
cursor.execute(query_group_by_product)
result_group_by_product = cursor.fetchall()

print("Conteo de pedidos por producto (GROUP BY):")
for row in result_group_by_product:
    print(row)

print("\n--------------------\n")

# Ejemplo de SUM o AVG: cantidad total pedida por producto
query_sum_quantity = """
SELECT product_id, SUM(quantity) AS total_quantity
FROM orders
GROUP BY product_id;
"""
cursor.execute(query_sum_quantity)
result_sum_quantity = cursor.fetchall()

print("Cantidad total pedida por producto (SUM):")
for row in result_sum_quantity:
    print(row)


Conteo de pedidos por producto (GROUP BY):
(101, 2)
(102, 1)
(103, 1)
(104, 1)
(105, 1)

--------------------

Cantidad total pedida por producto (SUM):
(101, 8)
(102, 2)
(103, 1)
(104, 2)
(105, 7)


In [8]:
# ------------------------------------------
# CELDA 8: JOIN entre las tablas
# ------------------------------------------

# Unimos orders y products para obtener nombres de productos en los pedidos
query_join = """
SELECT o.order_id, o.product_id, p.product_name, o.quantity, o.order_date, o.status
FROM orders o
JOIN products p
ON o.product_id = p.product_id;
"""
cursor.execute(query_join)
result_join = cursor.fetchall()

print("JOIN entre 'orders' y 'products':")
for row in result_join:
    print(row)


JOIN entre 'orders' y 'products':
(1, 101, 'Widget A', 3, '2023-01-10', 'shipped')
(2, 103, 'Thingamajig', 1, '2023-01-11', 'shipped')
(3, 102, 'Widget B', 2, '2023-01-12', 'cancelled')
(4, 101, 'Widget A', 5, '2023-01-15', 'shipped')
(5, 104, 'Gadget', 2, '2023-01-15', 'pending')
(6, 105, 'Machine Part', 7, '2023-01-18', 'shipped')


In [9]:
# ------------------------------------------
# CELDA 9: Subconsulta (subquery)
# ------------------------------------------

# Ejemplo: queremos todos los pedidos de los productos que pertenecen a 'Category A'
query_subquery = """
SELECT *
FROM orders
WHERE product_id IN (
    SELECT product_id
    FROM products
    WHERE category = 'Category A'
);
"""
cursor.execute(query_subquery)
result_subquery = cursor.fetchall()

print("Pedidos de productos en 'Category A':")
for row in result_subquery:
    print(row)


Pedidos de productos en 'Category A':
(1, 101, 3, '2023-01-10', 'shipped')
(3, 102, 2, '2023-01-12', 'cancelled')
(4, 101, 5, '2023-01-15', 'shipped')
(5, 104, 2, '2023-01-15', 'pending')
